In [101]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys
sys.path.append('/home/xilinx')
import numpy as np
from math import sqrt
from pynq import Overlay

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/r09525031/cordic/design_1_wrapper.bit")
    cordic = ol.cordic_0

    
    ## system is 14bits signed ap_fixed, 2bits int(1bit for sign) and 12s bit fixed 
    test_num = 90; ## due to input pattern and 2bit int 
    shifter = 4096.0; ##due to 12bit fixed
    gold_sin = 0.0;
    gold_cos = 0.0;
    sum_sin_error = 0.0;
    sum_cos_error = 0.0;
    
    
    for i in range(test_num):
        radian = i*PI/180.0;
        gold_sin = np.sin(radian);
        gold_cos = np.cos(radian);
        pat = int(radian*shifter);
        
        cordic.write(0x10, pat)
        cordic.write(0x00,0x01);
        out_sin = cordic.read(0x18)
        out_cos = cordic.read(0x20)
       
    ##sign check
        if(0x0 |(out_sin & 0x2000)>1):
            out_sin = -(out_sin & 0x1FFF);
        else:
            out_sin = out_sin & 0x1FFF;
        if(0x0 |(out_cos & 0x2000)>1):
            out_cos = -(out_cos & 0x1FFF);
        else:
            out_cos = out_cos & 0x1FFF;
        if(i==0): ##known issue, first data would read wrong
            continue;
        sum_sin_error += (gold_sin - out_sin/shifter) * (gold_sin - out_sin/shifter);
        sum_cos_error += (gold_cos - out_cos/shifter) * (gold_cos - out_cos/shifter);
        
    Rms_sin_error = sqrt(sum_sin_error/test_num);
    Rms_cos_error = sqrt(sum_cos_error/test_num);
    print("Rms_sin_error = " + str(Rms_sin_error));
    print("Rms_cos_error = " + str(Rms_cos_error));
    
    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Rms_sin_error = 0.00046632649383929696
Rms_cos_error = 0.0005111448200322633
Exit process
